In [167]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from fastapi import FastAPI
import uvicorn
from sklearn.metrics.pairwise import linear_kernel

In [168]:
df = pd.read_csv('DataSet.csv')

In [169]:
app = FastAPI()

In [170]:
new_df = df.copy()

In [171]:
new_df['Rating'].fillna(new_df['Rating'].mean(), inplace=True)
new_df['Review Count'].fillna(new_df['Review Count'].mean, inplace=True)
new_df['Prerequisites'].fillna("No Prerequisites Required",inplace=True)
new_df['Affiliates'].fillna("None",inplace=True)
new_df['Level'].fillna("beginner",inplace=True)

C:\Users\anany\AppData\Local\Temp\ipykernel_42496\353552095.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<bound method Series.mean of 0      1802.0
1       864.0
2      1126.0
3      2649.0
4      1212.0
        ...  
262       NaN
263       NaN
264       NaN
265       NaN
266       NaN
Name: Review Count, Length: 267, dtype: float64>' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_df['Review Count'].fillna(new_df['Review Count'].mean, inplace=True)


In [172]:
new_df.isnull().sum()

Title             0
Type              0
Description       6
Level             0
Duration          4
Rating            0
Review Count      0
Skills Covered    0
Prerequisites     0
Affiliates        0
URL               0
dtype: int64

In [173]:
new_df.head()

,Title,Type,Description,Level,Duration,Rating,Review Count,Skills Covered,Prerequisites,Affiliates,URL
0,Data Engineering with AWS,nanodegree,"Learn to design data models, build data wareho...",intermediate,4 Months,4.6,1802.0,"AWS Glue, Amazon S3, AWS Data Warehouse, Re...","Intermediate Python, intermediate SQL & comman...",None,https://www.udacity.com/course/data-engineer-n...
1,Product Manager,nanodegree,Envision and execute the development of indust...,beginner,4 Months,4.7,864.0,"Product Strategy, Product Design, Product Deve...",No Experience Required,None,https://www.udacity.com/course/product-manager...
2,C++,nanodegree,Get hands-on experience building five real-wor...,intermediate,4 Months,4.5,1126.0,"Data Structures & Algorithms, Memory Managemen...",Intermediate Programming,None,https://www.udacity.com/course/c-plus-plus-nan...
3,Business Analytics,nanodegree,Gain foundational data skills like analyzing d...,beginner,3 Months,4.8,2649.0,"Excel & Spreadsheets, SQL, Data Visualization,...",No Prerequisites Required,Mode,https://www.udacity.com/course/business-analyt...
4,Data Scientist,nanodegree,"Build effective machine learning models, run d...",advanced,4 Months,4.7,1212.0,"Machine Learning, Deep Learning, Software Engi...","Python, SQL & Statistics","Bertelsmann, Figure Eight, IBM Watson, Insight...",https://www.udacity.com/course/data-scientist-...


In [174]:
new_df_2 = new_df[['Title','Level','Prerequisites','Skills Covered']]
new_df_2.head()

,Title,Level,Prerequisites,Skills Covered
0,Data Engineering with AWS,intermediate,"Intermediate Python, intermediate SQL & comman...","AWS Glue, Amazon S3, AWS Data Warehouse, Re..."
1,Product Manager,beginner,No Experience Required,"Product Strategy, Product Design, Product Deve..."
2,C++,intermediate,Intermediate Programming,"Data Structures & Algorithms, Memory Managemen..."
3,Business Analytics,beginner,No Prerequisites Required,"Excel & Spreadsheets, SQL, Data Visualization,..."
4,Data Scientist,advanced,"Python, SQL & Statistics","Machine Learning, Deep Learning, Software Engi..."


In [175]:
vectorizer = TfidfVectorizer()
df_vector = vectorizer.fit_transform(new_df_2)

In [176]:
similarity = cosine_similarity(df_vector)

In [177]:
def recommend(user_preferences, new_df_2):
  new_df_2['features'] = new_df_2['Title'] + ' ' +  new_df_2['Level'] + ' ' +  new_df_2['Prerequisites'] + ' ' + new_df_2['Skills Covered']
  vectorizer = TfidfVectorizer(stop_words='english')
  course_matrix = vectorizer.fit_transform(new_df_2['features'])
  cosine_similarities = linear_kernel(course_matrix, course_matrix)
  user_course_index = new_df_2[new_df_2['Title'] == user_preferences]
  similarity_scores = list(enumerate(cosine_similarities[user_course_index]))
  sorted_courses = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
  top_n_recommendations = [(new_df_2.iloc[idx]['Title'], score) for idx, score in sorted_courses[1:]]
  return top_n_recommendations

print(recommend('HTML', new_df_2))  


[]


C:\Users\anany\AppData\Local\Temp\ipykernel_42496\1117219696.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_2['features'] = new_df_2['Title'] + ' ' +  new_df_2['Level'] + ' ' +  new_df_2['Prerequisites'] + ' ' + new_df_2['Skills Covered']
